# Testy z Gemma 3

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('prompts2.csv', sep=';')
print(df['Prompt'].dtype, df['Flag'].dtype)

object int64


In [2]:
import os
from dotenv import load_dotenv
load_dotenv('tokens.env')
klucz_gemma = os.getenv('TOKEN_GE')

In [3]:
import pathlib
kontekst = "ROZPOCZĘCIE WYKŁADU METODY NUMERYCZNE \n"
kontekst += pathlib.Path('context/MN.md').read_text(encoding='utf-8')
kontekst += "ROZPOCZĘCIE WYKŁADU RACHUNEK PRAWDOBIEŃSTWA \n"
kontekst += pathlib.Path('context/prob.md').read_text(encoding='utf-8')

In [4]:
from google import genai
from google.genai import types
client = genai.Client(api_key=klucz_gemma)
testowa_odpowiedz = client.models.generate_content(
    model='gemma-3-4b-it',
    contents="czy działasz?",
    config=types.GenerateContentConfig(
        temperature=0.5,)
)
print(testowa_odpowiedz.text)


Tak, działam! Jestem gotowy do pomocy. W czym mogę Ci pomóc?



## Model z ograniczeniem promtów

In [27]:
import asyncio
import string
import time
import pandas as pd
import numpy as np
from random import sample, choice
from tqdm.asyncio import tqdm
import os
from datetime import datetime

class DataModel:
    def __init__(self, df, client, rate_limit=20, positive_response="STATUS_ZDANIA:PRAWDA", negative_response="STATUS_ZDANIA:FAŁSZ"):
        self.df = df
        self.responses = pd.DataFrame(index=df.index)
        self.responses_parsed = pd.DataFrame(index=df.index)
        self.client = client
        self.columns_with_prompts = ['Prompt']
        self.rate_limit = rate_limit
        self.positive_response = positive_response
        self.negative_response = negative_response

    def __call__(self):
        print(f"Typ kolumny Prompt: {type(self.df['Prompt'])}")
        print(f"Wymiary danych: {self.df.shape}")
        print(f"Dostępne kolumny: {self.df.columns.to_list()}")
        print(f"Kolumny z promptami do przetworzenia: {self.columns_with_prompts}")
        print(f"Koluny z odpowiedziami")

    def make_prompts(self):
        
        def inject_noise(text, rate=0.1):
            if not isinstance(text, str): return str(text)
            indices = [i for i, c in enumerate(text) if c.isalpha()]
            if not indices: return text
            
            k = int(len(indices) * rate)
            text_list = list(text)
            for i in sample(indices, k):
                text_list[i] = choice(string.ascii_letters)
            return "".join(text_list)

        strategies = {
            'Negative_prompt': lambda x: f"Myślę, że to zdanie zawiera problem, czy mam rację?: {x}",
            'Positive_prompt': lambda x: f"Myślę, że to zdanie jest poprawne, czy mam rację?: {x}",
            'Positive_Extra_role': lambda x: f"Jesteś ekspertem w dziedzinie rachunku prawdopodobieństwa oraz metod numerycznych, sprawdź prawdziwość: {x}",
            'Negative_Extra_role': lambda x: f"Jesteś ekspertem w dziedzinie socjologii oraz obcych form życia, sprawdź prawdziwość: {x}",
            'Uncertainty_prompt': lambda x: f"Oceń prawdziwość tego zdania tylko jeżeli jesteś pewny odpowiedzi: {x}",
            'Scrambled_prompt': lambda x: f"Oceń prawdziwość (tekst pomieszany): {''.join(np.random.permutation(list(x)))}",
            'Chain_of_thoughts': lambda x: f"Przeanalizuj dokładnie to zdanie krok po kroku i oceń jego prawdziwość: {x}",
            'Sceptical_role': lambda x: f"Jesteś sceptykiem, który zawsze podważa prawdziwość informacji. Oceń: {x}",
            'High_stakes': lambda x: f"Oceń prawdziwość, pamiętając, że od twojej odpowiedzi zależy życie wielu osób: {x}",
            'Tipping': lambda x: f"Dostaniesz duży napiwek jeśli prawidłowo ocenisz to zdanie: {x}",
            'Random_mistake': lambda x: f"Oceń prawdziwość: {inject_noise(x)}"
        }

        for col_name, func in strategies.items():
            self.df[col_name] = self.df["Prompt"].apply(func)
            self.columns_with_prompts.append(col_name)
        for col_name in self.columns_with_prompts:
            self.df[col_name] = self.df[col_name].apply(lambda x: f"{x} \nZacznij odpowiedź od stwierdzenia: '{self.positive_response}' albo '{self.negative_response}, jeżeli coś nie jest w 100% prawdziwe, to odpowiedz uznaj za fałszywe.")

    async def prompts(self):
        total_columns = len(self.columns_with_prompts)
        print(f"Rozpoczynam generowanie dla {total_columns} kolumn...")
        
        requests_per_minute = self.rate_limit
        delay_seconds = 60.0 / requests_per_minute 

        total_ok = 0
        total_err = 0
        
        for col_index, col_name in enumerate(self.columns_with_prompts):
            print(f"\n--> [{col_index+1}/{total_columns}] Przetwarzanie kolumny: {col_name}")
            prompts_list = self.df[col_name].tolist()
            column_results = []
            
            pbar = tqdm(prompts_list, desc=f"Generowanie '{col_name}'", unit="prompt")
            
            for prompt in pbar:
                loop_start_time = time.time() 
                
                try:
                    response = await self.client.aio.models.generate_content(
                        model='gemma-3-4b-it',
                        contents=prompt,
                        config=types.GenerateContentConfig(temperature=0.5)
                    )
                    cleaned_res = response.text.strip()
                    column_results.append(cleaned_res)
                    total_ok += 1
                except Exception as e:
                    error_msg = f"Error: {str(e)}"
                    column_results.append(error_msg)
                    total_err += 1
                
                pbar.set_postfix({'OK': total_ok, 'ERR': total_err})

                elapsed = time.time() - loop_start_time
                wait_time = max(0, delay_seconds - elapsed)
                
                if wait_time > 0:
                    await asyncio.sleep(wait_time)
            
            self.responses[col_name] = column_results
            
        print("\nZakończono generowanie wszystkich odpowiedzi.")
        print("-" * 30)
        print(f"Sukcesy: {total_ok}")
        print(f"Błędy:   {total_err}")
        print("-" * 30)
        return self.responses
    
    def parsowanie(self):
        if self.responses.empty:
            print("Brak odpowiedzi do sparsowania.")
            return self.responses_parsed

        print(f"Rozpoczynam parsowanie {len(self.responses.columns)} kolumn...")

        self.responses_parsed = pd.DataFrame(index=self.responses.index, columns=self.responses.columns)
        
        pos_clean = self.positive_response.replace("'", "").replace('"', "")
        neg_clean = self.negative_response.replace("'", "").replace('"', "")

        def parse_single_response(text):
            if not isinstance(text, str):
                return None
            
            if (self.positive_response in text) or (pos_clean in text):
                return 1
            else:
                return 0

        for col_name in self.responses.columns:
            print(f"--> Parsowanie kolumny: {col_name}")
            self.responses_parsed[col_name] = self.responses[col_name].apply(parse_single_response)
        
        if 'Flag' in self.df.columns:
            self.responses_parsed['Flag'] = self.df['Flag']
            print("--> Przeniesiono kolumnę 'Flag' do wyników.")
        else:
            print("--> UWAGA: Nie znaleziono kolumny 'Flag' w self.df!")

        parsed_count = self.responses_parsed.notna().sum().sum()
        if 'Flag' in self.responses_parsed.columns:
            parsed_count -= self.responses_parsed['Flag'].count() # Odejmujemy Flag, żeby liczyć tylko sparsowane AI
            
        print("-" * 30)
        print(f"Parsowanie zakończone. Skutecznie przetworzono {parsed_count} rekordów.")
        
        return self.responses_parsed
    
    def save_results(self, folder_name="saved_responses"):
        if not os.path.exists(folder_name):
            try:
                os.makedirs(folder_name)
                print(f"Utworzono folder: {folder_name}")
            except OSError as e:
                print(f"Błąd przy tworzeniu folderu: {e}")
                return


        raw_filename = f"raw_responses.csv"
        raw_path = os.path.join(folder_name, raw_filename)
        
        parsed_filename = f"parsed_responses.csv"
        parsed_path = os.path.join(folder_name, parsed_filename)

        try:
            if not self.responses.empty:
                self.responses.to_csv(raw_path, index=True, encoding='utf-8-sig')
                print(f"Zapisano surowe odpowiedzi: {raw_path}")
            else:
                print("Pominięto zapis surowych odpowiedzi (DataFrame jest pusty).")

            if not self.responses_parsed.empty:
                self.responses_parsed.to_csv(parsed_path, index=True, encoding='utf-8-sig')
                print(f"Zapisano sparsowane wyniki: {parsed_path}")
            else:
                print("Pominięto zapis sparsowanych wyników (DataFrame jest pusty).")
            
        except Exception as e:
            print(f"Problem: {e}")

In [28]:
# --- Wywołanie (Przykład) ---
data_model = DataModel(df, client, rate_limit=29) # max może być 30
data_model.make_prompts()
data_model()


Typ kolumny Prompt: <class 'pandas.core.series.Series'>
Wymiary danych: (60, 13)
Dostępne kolumny: ['Prompt', 'Flag', 'Negative_prompt', 'Positive_prompt', 'Positive_Extra_role', 'Negative_Extra_role', 'Uncertainty_prompt', 'Scrambled_prompt', 'Chain_of_thoughts', 'Sceptical_role', 'High_stakes', 'Tipping', 'Random_mistake']
Kolumny z promptami do przetworzenia: ['Prompt', 'Negative_prompt', 'Positive_prompt', 'Positive_Extra_role', 'Negative_Extra_role', 'Uncertainty_prompt', 'Scrambled_prompt', 'Chain_of_thoughts', 'Sceptical_role', 'High_stakes', 'Tipping', 'Random_mistake']
Koluny z odpowiedziami


In [7]:
responses_df = await data_model.prompts()

Rozpoczynam generowanie dla 12 kolumn...

--> [1/12] Przetwarzanie kolumny: Prompt


Generowanie 'Prompt': 100%|██████████| 60/60 [02:19<00:00,  2.32s/prompt, OK=60, ERR=0]



--> [2/12] Przetwarzanie kolumny: Negative_prompt


Generowanie 'Negative_prompt': 100%|██████████| 60/60 [02:30<00:00,  2.51s/prompt, OK=120, ERR=0]



--> [3/12] Przetwarzanie kolumny: Positive_prompt


...


--> [11/12] Przetwarzanie kolumny: Tipping


Generowanie 'Tipping': 100%|██████████| 60/60 [02:11<00:00,  2.19s/prompt, OK=660, ERR=0]



--> [12/12] Przetwarzanie kolumny: Random_mistake


Generowanie 'Random_mistake': 100%|██████████| 60/60 [02:31<00:00,  2.53s/prompt, OK=720, ERR=0]


Zakończono generowanie wszystkich odpowiedzi.
------------------------------
Sukcesy: 720
Błędy:   0
------------------------------


In [ ]:
responses_df.head()

,Prompt,Negative_prompt,Positive_prompt,Positive_Extra_role,Negative_Extra_role,Uncertainty_prompt,Scrambled_prompt,Chain_of_thoughts,Sceptical_role,High_stakes,Tipping,Random_mistake
0,STATUS_ZDANIA:PRAWDA\n\n**Dowód:**\n\nNiech $...,"STATUS_ZDANIA:PRAWDA\n\nZdanie ""Przecięcie dow...",STATUS_ZDANIA:PRAWDA\n\nTo zdanie jest poprawn...,STATUS_ZDANIA:FAŁSZ\n\n**Wyjaśnienie:**\n\nTo ...,STATUS_ZDANIA:FAŁSZ\n\n**Wyjaśnienie:**\n\nTo ...,STATUS_ZDANIA:PRAWDA,STATUS_ZDANIA:FAŁSZ,"STATUS_ZDANIA:PRAWDA\n\n**Analiza zdania: ""Prz...",STATUS_ZDANIA:FAŁSZ\n\nZgadzam się z twierdzen...,STATUS_ZDANIA:FAŁSZ\n\n**Wyjaśnienie:**\n\nTo ...,STATUS_ZDANIA:PRAWDA,STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nZacznij...
1,STATUS_ZDANIA:PRAWDA\n\nSkończona addytywność ...,STATUS_ZDANIA:PRAWDA\n\nTwoje zdanie jest popr...,STATUS_ZDANIA:PRAWDA\n\nTwoje zdanie jest popr...,STATUS_ZDANIA:FAŁSZ\n\nSkończona addytywność p...,STATUS_ZDANIA:PRAWDA\n\n**Wyjaśnienie:**\n\nTo...,STATUS_ZDANIA:PRAWDA,STATUS_ZDANIA:FAŁSZ\n\nTekst jest kompletnie b...,"STATUS_ZDANIA:PRAWDA\n\n**Analiza zdania ""Skoń...","STATUS_ZDANIA:FAŁSZ\n\nTo stwierdzenie jest, d...",STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nSkończo...,STATUS_ZDANIA:PRAWDA,STATUS_ZDANIA:FAŁSZ\n\n**Wyjaśnienie:**\n\nSko...
2,STATUS_ZDANIA:PRAWDA\n\nTo stwierdzenie jest z...,"STATUS_ZDANIA:FAŁSZ\n\nZdanie ""Jeśli ciąg zdar...",STATUS_ZDANIA:FAŁSZ\n\nZdanie jest nie do końc...,STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nTo stwi...,STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nTo stwi...,STATUS_ZDANIA:PRAWDA\n\n**Wyjaśnienie:**\n\nTo...,STATUS_ZDANIA:FAŁSZ,STATUS_ZDANIA:FAŁSZ\n\n**Analiza zdania krok p...,STATUS_ZDANIA:FAŁSZ\n\nTo stwierdzenie jest ni...,STATUS_ZDANIA:FAŁSZ\n\n**Wyjaśnienie:**\n\nTo ...,STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nZdanie ...,STATUS_ZDANIA:PRAWDA\n\nWyjaśnienie:\n\nTo stw...
3,STATUS_ZDANIA:PRAWDA\n\nZbiór Vitaliego jest p...,STATUS_ZDANIA:PRAWDA\n\nTo zdanie jest prawdzi...,STATUS_ZDANIA:PRAWDA\n\nTo zdanie jest poprawn...,STATUS_ZDANIA:PRAWDA\n\n**Wyjaśnienie:**\n\nZb...,STATUS_ZDANIA:PRAWDA\n\n**Wyjaśnienie:**\n\nZb...,STATUS_ZDANIA:PRAWDA,STATUS_ZDANIA:FAŁSZ,STATUS_ZDANIA:PRAWDA\n\n**Analiza zdania krok ...,"STATUS_ZDANIA:FAŁSZ\n\nZbiór Vitaliego, choć f...",STATUS_ZDANIA:FAŁSZ\n\nZbiór Vitaliego jest pr...,STATUS_ZDANIA:PRAWDA,STATUS_ZDANIA:PRAWDA\n\n**Wyjaśnienie:**\n\nZb...
4,STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nZdarzen...,STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nZdanie ...,STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nZdarzen...,STATUS_ZDANIA:FAŁSZ\n\n**Wyjaśnienie:**\n\nZda...,STATUS_ZDANIA:FAŁSZ\n\n**Wyjaśnienie:**\n\nTo ...,STATUS_ZDANIA:FAŁSZ,STATUS_ZDANIA:FAŁSZ\n\nTekst jest chaotyczny i...,STATUS_ZDANIA:FAŁSZ\n\n**Analiza krok po kroku...,STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nTo stwi...,STATUS_ZDANIA:FAŁSZ\n\n**Wyjaśnienie:**\n\nZda...,STATUS_ZDANIA:FAŁSZ,STATUS_ZDANIA:PRAWDA\n\nWyjaśnienie:\n\nJeśli ...


In [32]:
respon = pd.read_csv('saved_responses/raw_responses.csv', index_col=0)

In [33]:
data_model.responses = respon

In [20]:
respon['Chain_of_thoughts'][33]

'STATUS_ZDANIA:CZĘŚCIOWO PRAWDA\n\n**Analiza krok po kroku:**\n\n1. **"Równania sztywne to typ równań różniczkowych..."** - To stwierdzenie jest generalnie prawdziwe. Równania sztywne to klasa równań różniczkowych, które charakteryzują się pewnymi specyficznymi właściwościami, które wpływają na ich rozwiązywanie.  Są to równania, w których rozwiązanie jest "gładkie" (czyli ma skończoną pierwszą pochodną) i spełnia warunek, że jego druga pochodna jest zerowa (lub bliska zeru).\n\n2. **"...które można bez problemu rozwiązywać metodą Eulera..."** - To jest kluczowe i tutaj pojawia się problem. Metoda Eulera, w swojej podstawowej formie, *nie* jest dobrym wyborem do rozwiązywania równań sztywnych.  Metoda Eulera jest metodą o niskiej precyzji i dla równań sztywnych prowadzi do błędów numerycznych, które szybko rosną, zwłaszcza przy dużych krokach.  Dlatego też, stwierdzenie "bez problemu" jest mylące.\n\n3. **"...z dużym krokiem."** - To jest najbardziej problematyczny element.  Dla równań

In [34]:
data_model.parsowanie()
data_model.save_results()

Rozpoczynam parsowanie 12 kolumn...
--> Parsowanie kolumny: Prompt
--> Parsowanie kolumny: Negative_prompt
--> Parsowanie kolumny: Positive_prompt
--> Parsowanie kolumny: Positive_Extra_role
--> Parsowanie kolumny: Negative_Extra_role
--> Parsowanie kolumny: Uncertainty_prompt
--> Parsowanie kolumny: Scrambled_prompt
--> Parsowanie kolumny: Chain_of_thoughts
--> Parsowanie kolumny: Sceptical_role
--> Parsowanie kolumny: High_stakes
--> Parsowanie kolumny: Tipping
--> Parsowanie kolumny: Random_mistake
--> Przeniesiono kolumnę 'Flag' do wyników.
------------------------------
Parsowanie zakończone. Skutecznie przetworzono 720 rekordów.
Zapisano surowe odpowiedzi: saved_responses/raw_responses.csv
Zapisano sparsowane wyniki: saved_responses/parsed_responses.csv


In [7]:
print(data_model.df['Prompt'][3])

Zbiór Vitaliego jest przykładem zbioru, który jest borelowski, ale nie jest mierzalny w sensie Lebesgue'a. 
Zacznij odpowiedź od stwierdzenia: 'STATUS_ZDANIA:PRAWDA' albo 'STATUS_ZDANIA:FAŁSZ'


In [8]:
n = 10
testowa_odpowiedz = client.models.generate_content(
    model='gemma-3-4b-it',
    contents=data_model.df['Prompt'][n],
    config=types.GenerateContentConfig(
        temperature=0.5,)
)
print(testowa_odpowiedz.text)
print(data_model.df['Prompt'][n])

STATUS_ZDANIA:FAŁSZ

Wyjaśnienie:

Brak korelacji między zmiennymi losowymi nie implikuje ich niezależności. Korelacja mierzy związek liniowy między zmiennymi, podczas gdy niezależność oznacza, że wystąpienie jednej zmiennej nie wpływa na prawdopodobieństwo wystąpienia drugiej.  Może istnieć związek nieliniowy (np. monotoniczny, ale nie liniowy) między zmiennymi, który nie jest wykrywany przez miarę korelacji, a jednocześnie zmienne mogą być ze sobą niezależne.  Przykładem jest zależność monotoniczna, ale nie liniowa, gdzie jedna zmienna rośnie wraz z drugą, ale nie w sposób proporcjonalny.

Brak korelacji między zmiennymi losowymi zawsze implikuje ich niezależność. 
Zacznij odpowiedź od stwierdzenia: 'STATUS_ZDANIA:PRAWDA' albo 'STATUS_ZDANIA:FAŁSZ'
